# 🚀 dLNk GPT Agent v2 - Phase 2: Long-form Code Generation\n\n**เป้าหมาย:** Fine-tune โมเดล dLNk-gpt-j-6b-agent-v1 ให้มีความสามารถในการเขียนโค้ดยาวๆ ได้ดีขึ้น\n\n**Training Data:**\n- CodeAlpaca-20k (~20,000 examples)\n- Python Code Instructions 18k (~18,000 examples)\n- Code Instructions 120k subset (~20,000 examples)\n- **รวม ~58,000 examples**\n\n**ระยะเวลา:** 10-15 ชั่วโมง\n\n**GPU:** A100 High-RAM\n\n---\n\n## 📝 วิธีใช้งาน\n\n1. **ตั้งค่า GPU:** Runtime > Change runtime type > A100 GPU\n2. **รัน Cell 1:** ติดตั้ง dependencies\n3. **Restart Runtime:** Runtime > Restart runtime (สำคัญมาก!)\n4. **รัน Cell 2 เป็นต้นไป:** ข้าม Cell 1\n\n---

In [ ]:
1: ติดตั้ง Dependencies (ใช้เวอร์ชันล่าสุดที่เข้ากันได้)
# ================================================================

!pip install -q -U pip setuptools wheel

# ติดตั้ง packages โดยไม่ระบุเวอร์ชัน ให้ pip จัดการ dependencies เอง
!pip install -q -U transformers accelerate peft bitsandbytes datasets trl huggingface_hub wandb requests

print("✅ ติดตั้ง dependencies เสร็จสมบูรณ์")
print("⚠️ กรุณา Restart Runtime: Runtime > Restart runtime")
print("⚠️ หลังจาก restart แล้ว ให้รัน Cell 2 เป็นต้นไป (ข้าม Cell 1)")

In [ ]:
2: Configuration และ Tokens
# ================================================================

import os
from datetime import datetime

# ==================== กรุณากรอกข้อมูลที่นี่ ====================

# Hugging Face Token (ต้องมีสิทธิ์ write)
HF_TOKEN = ""  # กรอก HF Token ของคุณที่นี่  # กรอก HF Token ของคุณที่นี่

# Telegram Bot Configuration
TELEGRAM_BOT_TOKEN = ""  # กรอก Telegram Bot Token ที่นี่  # กรอก Telegram Bot Token ที่นี่
TELEGRAM_CHAT_ID = ""  # กรอก Telegram Chat ID ที่นี่    # กรอก Telegram Chat ID ที่นี่

# Weights & Biases (Optional - ถ้าไม่ใช้ให้เว้นว่าง)
WANDB_API_KEY = ""       # กรอก W&B API Key ที่นี่ (optional)
WANDB_PROJECT = "dLNk-gpt-v2"
WANDB_RUN_NAME = f"phase2-longform-{datetime.now().strftime('%Y%m%d-%H%M')}"

# Model Configuration
BASE_MODEL = "dlnkgpt/dLNk-gpt-j-6b-agent-v1"  # โมเดล v1 ที่เทรนไว้แล้ว
NEW_MODEL_NAME = "dlnkgpt/dLNk-gpt-j-6b-agent-v2-phase2"

# Training Configuration
MAX_SEQ_LENGTH = 2048  # เพิ่มจาก 1024 เป็น 2048 เพื่อรองรับโค้ดยาว
NUM_TRAIN_EPOCHS = 3
BATCH_SIZE = 1
GRADIENT_ACCUMULATION_STEPS = 16
LEARNING_RATE = 2e-5
WARMUP_STEPS = 100
LOGGING_STEPS = 50
SAVE_STEPS = 500

# ================================================================

# ตั้งค่า environment variables
os.environ["HF_TOKEN"] = HF_TOKEN
if WANDB_API_KEY:
    os.environ["WANDB_API_KEY"] = WANDB_API_KEY
    os.environ["WANDB_PROJECT"] = WANDB_PROJECT

print("✅ Configuration เสร็จสมบูรณ์")
print(f"📦 Base Model: {BASE_MODEL}")
print(f"🎯 New Model: {NEW_MODEL_NAME}")
print(f"📏 Max Sequence Length: {MAX_SEQ_LENGTH}")
print(f"🔄 Epochs: {NUM_TRAIN_EPOCHS}")

In [ ]:
3: Telegram Notification Function
# ================================================================

import requests
from datetime import datetime

def send_telegram_message(message):
    """ส่งข้อความไปยัง Telegram"""
    if not TELEGRAM_BOT_TOKEN or not TELEGRAM_CHAT_ID:
        print("⚠️ Telegram not configured")
        return
    
    try:
        url = f"https://api.telegram.org/bot{TELEGRAM_BOT_TOKEN}/sendMessage"
        data = {
            "chat_id": TELEGRAM_CHAT_ID,
            "text": message,
            "parse_mode": "Markdown"
        }
        response = requests.post(url, data=data)
        if response.status_code == 200:
            print("✅ Telegram notification sent")
        else:
            print(f"⚠️ Telegram error: {response.status_code}")
    except Exception as e:
        print(f"❌ Telegram error: {e}")

# ทดสอบ Telegram
send_telegram_message(
    f"🚀 *dLNk GPT Agent v2 - Phase 2*\\n\\n"
    f"📅 เริ่มการเทรน: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\\n"
    f"🎯 เป้าหมาย: Long-form Code Generation\\n"
    f"⏱️ ระยะเวลาโดยประมาณ: 10-15 ชั่วโมง"
)

print("✅ Telegram notification function ready")

In [ ]:
4: โหลด Base Model และ Tokenizer
# ================================================================

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel, prepare_model_for_kbit_training

print("📥 กำลังโหลด base model...")

# ตั้งค่า quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# โหลด tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    "EleutherAI/gpt-j-6b",
    trust_remote_code=True
)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# โหลด base model
base_model = AutoModelForCausalLM.from_pretrained(
    "EleutherAI/gpt-j-6b",
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

# โหลด LoRA adapter จาก v1
print("📥 กำลังโหลด LoRA adapter จาก v1...")
model = PeftModel.from_pretrained(
    base_model,
    BASE_MODEL,
    token=HF_TOKEN
)

# เตรียมโมเดลสำหรับการเทรนต่อ
model = prepare_model_for_kbit_training(model)

print("✅ โหลดโมเดล v1 เสร็จสมบูรณ์")
print(f"🔧 Device: {model.device}")
print(f"💾 Model size: {model.get_memory_footprint() / 1e9:.2f} GB")

send_telegram_message(
    f"✅ *โหลดโมเดล v1 สำเร็จ*\\n\\n"
    f"📦 Base: EleutherAI/gpt-j-6b\\n"
    f"🔧 LoRA: {BASE_MODEL}\\n"
    f"💾 Memory: {model.get_memory_footprint() / 1e9:.2f} GB"
)

In [ ]:
5: เตรียม Training Data (ใช้ datasets ที่มีจริง)
# ================================================================

from datasets import load_dataset
import random

print("📥 กำลังโหลด training datasets...")

# ==================== Dataset 1: CodeAlpaca-20k (มีจริงและใช้งานได้) ====================
print("📥 Loading CodeAlpaca-20k...")

try:
    code_alpaca = load_dataset("sahil2801/CodeAlpaca-20k", split="train")
    
    # แปลงเป็น format ที่ต้องการ
    code_alpaca_data = []
    for example in code_alpaca:
        code_alpaca_data.append({
            "instruction": example.get("instruction", ""),
            "input": example.get("input", ""),
            "output": example.get("output", "")
        })
    
    print(f"✅ Loaded {len(code_alpaca_data)} examples from CodeAlpaca-20k")
    
except Exception as e:
    print(f"⚠️ Error loading CodeAlpaca-20k: {e}")
    code_alpaca_data = []

# ==================== Dataset 2: Python Code Instructions 18k ====================
print("📥 Loading Python Code Instructions 18k...")

try:
    python_code = load_dataset("iamtarun/python_code_instructions_18k_alpaca", split="train")
    
    python_code_data = []
    for example in python_code:
        python_code_data.append({
            "instruction": example.get("instruction", ""),
            "input": example.get("input", ""),
            "output": example.get("output", "")
        })
    
    print(f"✅ Loaded {len(python_code_data)} examples from Python Code Instructions")
    
except Exception as e:
    print(f"⚠️ Error loading Python Code Instructions: {e}")
    python_code_data = []

# ==================== Dataset 3: Code Instructions 120k (เอาบางส่วน) ====================
print("📥 Loading Code Instructions 120k (subset)...")

try:
    code_instructions = load_dataset("iamtarun/code_instructions_120k_alpaca", split="train[:20000]")  # เอาแค่ 20k
    
    code_instructions_data = []
    for example in code_instructions:
        code_instructions_data.append({
            "instruction": example.get("instruction", ""),
            "input": example.get("input", ""),
            "output": example.get("output", "")
        })
    
    print(f"✅ Loaded {len(code_instructions_data)} examples from Code Instructions")
    
except Exception as e:
    print(f"⚠️ Error loading Code Instructions: {e}")
    code_instructions_data = []

# ==================== รวม Datasets ====================
all_data = code_alpaca_data + python_code_data + code_instructions_data

# Shuffle
random.shuffle(all_data)

print(f"\\n📊 Total training examples: {len(all_data)}")

# กรองเฉพาะข้อมูลที่มี output ยาวพอ (สำหรับ long-form code generation)
filtered_data = [d for d in all_data if len(d["output"]) >= 100]  # อย่างน้อย 100 ตัวอักษร

print(f"📊 Filtered examples (long code): {len(filtered_data)}")

# แบ่ง train/validation
train_size = int(0.9 * len(filtered_data))
train_data = filtered_data[:train_size]
val_data = filtered_data[train_size:]

print(f"📊 Train: {len(train_data)} examples")
print(f"📊 Validation: {len(val_data)} examples")

# สร้าง Dataset object
from datasets import Dataset

train_dataset = Dataset.from_list(train_data)
val_dataset = Dataset.from_list(val_data)

print("✅ Training data พร้อมใช้งาน")

send_telegram_message(
    f"✅ *เตรียมข้อมูลเสร็จสิ้น*\\n\\n"
    f"📊 Training examples: {len(train_data):,}\\n"
    f"📊 Validation examples: {len(val_data):,}\\n"
    f"📏 Max sequence length: {MAX_SEQ_LENGTH}"
)

In [ ]:
6: Format Data และ Tokenize
# ================================================================

def format_instruction(example):
    """Format data เป็น instruction format"""
    instruction = example["instruction"]
    input_text = example.get("input", "")
    output = example["output"]
    
    if input_text:
        prompt = f"### Instruction:\\n{instruction}\\n\\n### Input:\\n{input_text}\\n\\n### Response:\\n"
    else:
        prompt = f"### Instruction:\\n{instruction}\\n\\n### Response:\\n"
    
    return prompt + output + tokenizer.eos_token

def tokenize_function(examples):
    """Tokenize ข้อมูล"""
    text = format_instruction(examples)
    tokenized = tokenizer(
        text,
        truncation=True,
        max_length=MAX_SEQ_LENGTH,
        padding="max_length",
        return_tensors=None  # ไม่ใช้ tensors ใน map function
    )
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

print("🔄 กำลัง tokenize ข้อมูล...")

# Tokenize datasets
train_dataset = train_dataset.map(
    tokenize_function,
    remove_columns=train_dataset.column_names,
    desc="Tokenizing train dataset"
)

val_dataset = val_dataset.map(
    tokenize_function,
    remove_columns=val_dataset.column_names,
    desc="Tokenizing validation dataset"
)

print("✅ Tokenization เสร็จสมบูรณ์")

In [ ]:
7: ตั้งค่า Training Arguments
# ================================================================

from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./dLNk-gpt-v2-phase2",
    num_train_epochs=NUM_TRAIN_EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    learning_rate=LEARNING_RATE,
    warmup_steps=WARMUP_STEPS,
    logging_steps=LOGGING_STEPS,
    save_steps=SAVE_STEPS,
    evaluation_strategy="steps",
    eval_steps=SAVE_STEPS,
    save_total_limit=3,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    fp16=True,
    optim="paged_adamw_8bit",
    report_to="wandb" if WANDB_API_KEY else "none",
    run_name=WANDB_RUN_NAME if WANDB_API_KEY else None,
    logging_dir="./logs",
    remove_unused_columns=False,
)

print("✅ Training arguments ตั้งค่าเสร็จสมบูรณ์")
print(f"🔄 Epochs: {NUM_TRAIN_EPOCHS}")
print(f"📦 Batch size: {BATCH_SIZE}")
print(f"📈 Gradient accumulation: {GRADIENT_ACCUMULATION_STEPS}")
print(f"📊 Effective batch size: {BATCH_SIZE * GRADIENT_ACCUMULATION_STEPS}")
print(f"🎯 Learning rate: {LEARNING_RATE}")

In [ ]:
8: Custom Trainer with Telegram Notifications
# ================================================================

from transformers import Trainer, TrainerCallback

class TelegramCallback(TrainerCallback):
    """Callback สำหรับส่ง Telegram notifications"""
    
    def __init__(self):
        self.start_time = datetime.now()
        self.last_loss = None
    
    def on_log(self, args, state, control, logs=None, **kwargs):
        """เรียกทุกครั้งที่ log"""
        if logs and "loss" in logs:
            self.last_loss = logs["loss"]
            
            # ส่ง notification ทุก 50 steps
            if state.global_step % 50 == 0:
                elapsed = datetime.now() - self.start_time
                total_steps = state.max_steps
                progress = (state.global_step / total_steps) * 100
                
                send_telegram_message(
                    f"📊 *Step {state.global_step}/{total_steps}*\\n\\n"
                    f"📉 Loss: {logs['loss']:.4f}\\n"
                    f"📈 Progress: {progress:.1f}%\\n"
                    f"⏱️ Elapsed: {str(elapsed).split('.')[0]}"
                )
    
    def on_evaluate(self, args, state, control, metrics=None, **kwargs):
        """เรียกทุกครั้งที่ evaluate"""
        if metrics:
            send_telegram_message(
                f"📊 *Evaluation at Step {state.global_step}*\\n\\n"
                f"📉 Eval Loss: {metrics.get('eval_loss', 'N/A'):.4f}\\n"
                f"🎯 Train Loss: {self.last_loss:.4f}" if self.last_loss else ""
            )
    
    def on_train_end(self, args, state, control, **kwargs):
        """เรียกเมื่อเทรนเสร็จ"""
        elapsed = datetime.now() - self.start_time
        send_telegram_message(
            f"✅ *การเทรน Phase 2 เสร็จสิ้น!*\\n\\n"
            f"⏱️ เวลาทั้งหมด: {str(elapsed).split('.')[0]}\\n"
            f"📊 Total steps: {state.global_step}\\n"
            f"📉 Final loss: {self.last_loss:.4f}" if self.last_loss else ""
        )

# สร้าง Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    callbacks=[TelegramCallback()]
)

print("✅ Trainer พร้อมใช้งาน")

In [ ]:
9: เริ่มการเทรน
# ================================================================

print("🚀 เริ่มการเทรน Phase 2...")
print(f"⏰ เวลาเริ่มต้น: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

send_telegram_message(
    f"🚀 *เริ่มการเทรน Phase 2*\\n\\n"
    f"⏰ {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\\n"
    f"📊 Training examples: {len(train_dataset):,}\\n"
    f"🔄 Epochs: {NUM_TRAIN_EPOCHS}\\n"
    f"⏱️ ระยะเวลาโดยประมาณ: 10-15 ชั่วโมง"
)

# เริ่มเทรน
trainer.train()

print("✅ การเทรนเสร็จสมบูรณ์!")

In [ ]:
10: บันทึกและอัปโหลดโมเดล
# ================================================================

print("💾 กำลังบันทึกโมเดล...")

# บันทึกโมเดล
model.save_pretrained("./dLNk-gpt-v2-phase2-final")
tokenizer.save_pretrained("./dLNk-gpt-v2-phase2-final")

print("✅ บันทึกโมเดลเสร็จสมบูรณ์")

send_telegram_message(
    f"💾 *กำลังอัปโหลดโมเดล*\\n\\n"
    f"📦 Repository: {NEW_MODEL_NAME}\\n"
    f"⏳ กรุณารอสักครู่..."
)

# อัปโหลดไปยัง Hugging Face
print("📤 กำลังอัปโหลดโมเดลไปยัง Hugging Face...")

try:
    model.push_to_hub(
        NEW_MODEL_NAME,
        token=HF_TOKEN,
        commit_message=f"Phase 2: Long-form Code Generation - {datetime.now().strftime('%Y-%m-%d %H:%M')}"
    )
    tokenizer.push_to_hub(
        NEW_MODEL_NAME,
        token=HF_TOKEN
    )
    
    print(f"✅ อัปโหลดสำเร็จ: https://huggingface.co/{NEW_MODEL_NAME}")
    
    send_telegram_message(
        f"✅ *อัปโหลดโมเดลสำเร็จ!*\\n\\n"
        f"🔗 https://huggingface.co/{NEW_MODEL_NAME}\\n\\n"
        f"🎉 Phase 2 เสร็จสมบูรณ์!"
    )
    
except Exception as e:
    print(f"❌ เกิดข้อผิดพลาดในการอัปโหลด: {e}")
    
    send_telegram_message(
        f"⚠️ *เกิดข้อผิดพลาดในการอัปโหลด*\\n\\n"
        f"❌ Error: {str(e)}\\n\\n"
        f"💾 โมเดลถูกบันทึกไว้ที่: ./dLNk-gpt-v2-phase2-final"
    )

print("🎉 Phase 2 เสร็จสมบูรณ์!")

In [ ]:
11: ทดสอบโมเดล (Optional)
# ================================================================

print("🧪 ทดสอบโมเดล...")

test_prompt = "### Instruction:\\nWrite a comprehensive Python script for SQL injection testing:\\n\\n### Response:\\n"

inputs = tokenizer(test_prompt, return_tensors="pt").to(model.device)

outputs = model.generate(
    **inputs,
    max_new_tokens=512,
    temperature=0.7,
    do_sample=True,
    top_p=0.9
)

response = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("\\n" + "="*80)
print("🧪 Test Generation:")
print("="*80)
print(response)
print("="*80)

send_telegram_message(
    f"🧪 *ทดสอบโมเดล*\\n\\n"
    f"✅ โมเดลสามารถ generate โค้ดได้\\n"
    f"📝 ตัวอย่างถูกส่งไปแล้ว"
)

print("\\n✅ ทุกอย่างเสร็จสมบูรณ์!")